# Download and exctact the Dataset

In [10]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7xs7gMsLm6T0C4TipEtla9gZTiLzDxA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V7xs7gMsLm6T0C4TipEtla9gZTiLzDxA" -O data.zip && rm -rf /tmp/cookies.txt

--2022-11-15 12:22:25--  https://docs.google.com/uc?export=download&confirm=t&id=1V7xs7gMsLm6T0C4TipEtla9gZTiLzDxA
Resolving docs.google.com (docs.google.com)... 142.251.2.101, 142.251.2.113, 142.251.2.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nmfb8obvaggjfgemvck00klas24n0q89/1668514875000/17724475218651015314/*/1V7xs7gMsLm6T0C4TipEtla9gZTiLzDxA?e=download&uuid=7c6317b8-febc-43e1-b2ca-3de5b8ed1d1b [following]
--2022-11-15 12:22:25--  https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nmfb8obvaggjfgemvck00klas24n0q89/1668514875000/17724475218651015314/*/1V7xs7gMsLm6T0C4TipEtla9gZTiLzDxA?e=download&uuid=7c6317b8-febc-43e1-b2ca-3de5b8ed1d1b
Resolving doc-14-84-docs.googleusercontent.com (doc-14-84-docs.googleusercontent.com)... 142.251.2.132, 2607:

In [11]:
!unzip data.zip

Archive:  data.zip
  inflating: Set(08,56).bin          


In [12]:
!rm data.zip

In [13]:
!ls -lah

total 4.2G
drwxr-xr-x 1 root root 4.0K Nov 15 12:23  .
drwxr-xr-x 1 root root 4.0K Nov 15 10:04  ..
drwxr-xr-x 4 root root 4.0K Nov 11 14:31  .config
drwxr-xr-x 1 root root 4.0K Nov 11 14:32  sample_data
-rw-r--r-- 1 root root 4.2G Oct 26 01:24 'Set(08,56).bin'


# Import libraries and define functions

In [ ]:
import numpy as np
import pandas as pd
import pickle
import gc

In [ ]:
def getGroupRatios(df):
  return (df.j_tp + df.j_fp + df.j_tn + df.j_fn) / (df.i_tp + df.i_fp + df.i_tn + df.i_fn + df.j_tp + df.j_fp + df.j_tn + df.j_fn)

def getImbalanceRatios(df):
    return (df.i_tp + df.i_fn + df.j_tp + df.j_fn) / (df.i_tp + df.i_fp + df.i_tn + df.i_fn + df.j_tp + df.j_fp + df.j_tn + df.j_fn)

def getTPR_i(df):
    return df.i_tp / (df.i_tp + df.i_fn)

def getTPR_j(df):
    return df.j_tp / (df.j_tp + df.j_fn)

def getFPR_i(df):
    return df.i_fp / (df.i_fp + df.i_tn)

def getFPR_j(df):
    return df.j_fp / (df.j_fp + df.j_tn)

def getPositivePredictiveValue_i(df):
    return df.i_tp / (df.i_tp + df.i_fp)

def getPositivePredictiveValue_j(df):
    return df.j_tp / (df.j_tp + df.j_fp)

def getNegativePredictiveValue_i(df):
    return df.i_tn / (df.i_tn + df.i_fn)

def getNegativePredictiveValue_j(df):
    return df.j_tn / (df.j_tn + df.j_fn)

In [ ]:
dataCols = ['i_tp', 'i_fp', 'i_tn', 'i_fn', 'j_tp', 'j_fp', 'j_tn', 'j_fn']

# Write calculations of the 1st half of the dataset
As the dataset is quite large (4.2 Gb) we will write calculations in separate files in 2 stages.

In [ ]:
# Get half of the data
with open("Set(08,56).bin", "rb") as f:
    df = pd.DataFrame(pickle.load(f), columns = dataCols)

halfIdx = int(df.shape[0]/2)
df = df.iloc[:halfIdx]
df.head()

,i_tp,i_fp,i_tn,i_fn,j_tp,j_fp,j_tn,j_fn
0,56,0,0,0,0,0,0,0
1,55,1,0,0,0,0,0,0
2,55,0,1,0,0,0,0,0
3,55,0,0,1,0,0,0,0
4,55,0,0,0,1,0,0,0


In [ ]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276635335 entries, 0 to 276635334
Data columns (total 8 columns):
 #   Column  Dtype
---  ------  -----
 0   i_tp    int8 
 1   i_fp    int8 
 2   i_tn    int8 
 3   i_fn    int8 
 4   j_tp    int8 
 5   j_fp    int8 
 6   j_tn    int8 
 7   j_fn    int8 
dtypes: int8(8)
memory usage: 2.1 GB


In [ ]:
# Calculate half of GRs
with open("gr.bin", "wb+") as f:
    getGroupRatios(df).to_numpy().tofile(f)

# Calculate half of IRs
with open("ir.bin", "wb+") as f:
    getImbalanceRatios(df).to_numpy().tofile(f)

# Calculate half of Matrix Measures
with open("i_tpr.bin", "wb+") as f:
    getTPR_i(df).to_numpy().tofile(f)

with open("j_tpr.bin", "wb+") as f:
    getTPR_j(df).to_numpy().tofile(f)

with open("i_fpr.bin", "wb+") as f:
    getFPR_i(df).to_numpy().tofile(f)

with open("j_fpr.bin", "wb+") as f:
    getFPR_j(df).to_numpy().tofile(f)

with open("i_ppv.bin", "wb+") as f:
    getPositivePredictiveValue_i(df).to_numpy().tofile(f)

with open("j_ppv.bin", "wb+") as f:
    getPositivePredictiveValue_j(df).to_numpy().tofile(f)

with open("i_npv.bin", "wb+") as f:
    getNegativePredictiveValue_i(df).to_numpy().tofile(f)

with open("j_npv.bin", "wb+") as f:
    getNegativePredictiveValue_j(df).to_numpy().tofile(f)

# Append calculations of the 2st half of the dataset

In [ ]:
# Free the memory
del df
gc.collect()
gc.get_stats()

[{'collections': 374, 'collected': 3620, 'uncollectable': 0},
 {'collections': 36, 'collected': 300, 'uncollectable': 0},
 {'collections': 4, 'collected': 99, 'uncollectable': 0}]

In [ ]:
# Get half of the data
with open("Set(08,56).bin", "rb") as f:
    df = pd.DataFrame(pickle.load(f), columns = dataCols)

halfIdx = int(df.shape[0]/2)
df = df.iloc[halfIdx:]

In [ ]:
# Calculate half of GRs
with open("gr.bin", "ab+") as f:
    getGroupRatios(df).to_numpy().tofile(f)

# Calculate half of IRs
with open("ir.bin", "ab+") as f:
    getImbalanceRatios(df).to_numpy().tofile(f)

# Calculate half of Matrix Measures
with open("i_tpr.bin", "ab+") as f:
    getTPR_i(df).to_numpy().tofile(f)

with open("j_tpr.bin", "ab+") as f:
    getTPR_j(df).to_numpy().tofile(f)

with open("i_fpr.bin", "ab+") as f:
    getFPR_i(df).to_numpy().tofile(f)

with open("j_fpr.bin", "ab+") as f:
    getFPR_j(df).to_numpy().tofile(f)

with open("i_ppv.bin", "ab+") as f:
    getPositivePredictiveValue_i(df).to_numpy().tofile(f)

with open("j_ppv.bin", "ab+") as f:
    getPositivePredictiveValue_j(df).to_numpy().tofile(f)

with open("i_npv.bin", "ab+") as f:
    getNegativePredictiveValue_i(df).to_numpy().tofile(f)

with open("j_npv.bin", "ab+") as f:
    getNegativePredictiveValue_j(df).to_numpy().tofile(f)

# Compress and download the calculations
The files are written as `float64`, while the dataset has data-type `int8`, thus the size of each file is the same as the size of the dataset because each of them contains one 8 times heavier column.

In [ ]:
!ls -lah

total 46G
drwxr-xr-x 1 root root 4.0K Nov 15 12:18  .
drwxr-xr-x 1 root root 4.0K Nov 15 10:04  ..
drwxr-xr-x 4 root root 4.0K Nov 11 14:31  .config
-rw-r--r-- 1 root root 4.2G Nov 15 11:36  gr.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  i_fpr.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  i_npv.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  i_ppv.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:36  ir.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:36  i_tpr.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  j_fpr.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  j_npv.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  j_ppv.bin
-rw-r--r-- 1 root root 4.2G Nov 15 11:37  j_tpr.bin
drwxr-xr-x 1 root root 4.0K Nov 11 14:32  sample_data
-rw-r--r-- 1 root root 4.2G Oct 26 01:24 'Set(08,56).bin'


In [ ]:
!zip calculations.zip gr.bin ir.bin i_* j_*

updating: gr.bin (deflated 100%)
updating: ir.bin (deflated 97%)
updating: i_fpr.bin (deflated 100%)
updating: i_npv.bin (deflated 100%)
updating: i_ppv.bin (deflated 100%)
updating: i_tpr.bin (deflated 100%)
updating: j_fpr.bin (deflated 97%)
updating: j_npv.bin (deflated 98%)
updating: j_ppv.bin (deflated 95%)
updating: j_tpr.bin (deflated 96%)


In [ ]:
from google.colab import files
files.download("calculations.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>